IMPORT SECTION

In [15]:
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import numpy as np

In [16]:
import csv
import numpy as np
from sklearn.preprocessing import  StandardScaler
from sklearn.model_selection import train_test_split
from keras import layers
from keras import models
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
import xgboost as xgb

TRAINING SET LOADING

In [17]:
import csv
data=[]
labels=[]
with open("../FakeNewsNet/fake_with_feat.csv") as f:
    reader = csv.reader(f)

    conto=0
    for row in reader:
        l=[row[el] for el in range(len(row)-1)]
        data.append(l)
        labels.append(0)
        if(int(float(row[-1]))==-1):
            conto+=1
            if(conto==14000):
                break

with open("../FakeNewsNet/real_with_feat.csv") as f:
    reader = csv.reader(f)

    conto=0
    for row in reader:
        l=[row[el] for el in range(len(row)-1)]       
        data.append(l)
        labels.append(int(float(row[-1])))
        if(int(float(row[-1]))==1):
            conto+=1
            if(conto==14000):
                break



print(len(data), len(labels))
data = np.array(data, dtype='f')
labels = np.array(labels, dtype=int)

data, labels = shuffle(data, labels, random_state=42)

28000 28000


TRAINING AND VALIDATION SPLIT

In [18]:
#cretae module to load a dataset and prepare train and validation set
X_train, X_val, y_train, y_val = train_test_split(data, labels, random_state=42, test_size=0.3)
print(len(X_train), len(X_val), len(y_train), len(y_val))

19600 8400 19600 8400


TEST SET LOADING

In [20]:
data1=[]
labels1=[]
with open("../test.csv") as f:
        reader = csv.reader(f)

        conto=0
        for row in reader:
            if len(row) ==0:
                continue
            
            data1.append(row[:-1])
            if(int(float(row[-1]))==-1):
              labels1.append(0)
            else:
              labels1.append(int(float(row[-1])))
            if(int(float(row[-1]))==1):
                conto+=1
                if(conto==1259):
                    break

data1 = np.array(data1, dtype='f')
labels1 = np.array(labels1, dtype=int)

data1, labels1 = shuffle(data1, labels1, random_state=42)
print(data1.shape, labels1.shape)

(2518, 57) (2518,)


XGBOOST TRAINING FRAMEWORK

In [22]:
n_estimators=np.arange(100, 200, 50)
learning_rate=np.arange(0.1, 1, 0.1)
base_score=np.arange(0.1, 0.5, 0.1)
max_depth=np.arange(1, 10, 1)
subsample=np.arange(0.1, 1, 0.1)

dizionario={}
j=0
for n in n_estimators:
    for lr in learning_rate:
      for b in base_score:
        for m in max_depth:
            for samp in subsample:
              dizionario[j]=(n, lr, b, m, samp)
              j+=1
print(len(dizionario))

5832


In [ ]:
import xgboost as xgb
accuracy1=[]
val_accuracy=[]
val_covs=[]
covs1=[]

i=0
for n in n_estimators:
    for lr in learning_rate:
      for b in base_score:
        for m in max_depth:
            for samp in subsample:
              model = xgb.XGBClassifier(n_estimators = n, learning_rate=lr, base_score=b, max_depth=m, subsample=samp).fit(X_train, y_train)
              val_preds=model.predict(X_val)
              preds1=model.predict(data1)

              val_accuracy.append(balanced_accuracy_score(y_val, val_preds))
              accuracy1.append(balanced_accuracy_score(labels1, preds1))
              val_covs.append(confusion_matrix(y_val, val_preds))
              covs1.append(confusion_matrix(labels1, preds1))
              i+=1
              if(i%10==0):
                print(i)

val_covs=np.array(val_covs)
val_accuracy=np.array(val_accuracy)
covs1=np.array(covs1)
accuracy1=np.array(accuracy1)

BEST BOOSTER SELECTION

In [24]:
for i in range(len(accuracy1)):
    if(accuracy1[i]>0.6):
        if(val_accuracy[i]>0.7):
            print(accuracy1[i], val_accuracy[i], i)
            print(covs1[i], val_covs[i])

RECOMPUTE PERFORMANCES

In [8]:
#model = xgb.XGBClassifier(n_estimators = 1000, learning_rate=0.1, base_score=0.1, max_depth=7, subsample=1.5, colsample_bytree=0.8).fit(X_train, y_train)
model = xgb.XGBClassifier(n_estimators=100,base_score=0.9,learning_rate= 0.01,max_depth=15).fit(X_train, y_train)
#preds = model.predict(data)
val_preds=model.predict(X_val)
preds1= model.predict(data1)

for i in range(len(preds1)):
    preds1[i] =0 if preds1[i]<0.5 else 1

for i in range(len(val_preds)):
    val_preds[i] = 0 if val_preds[i]<0.5 else 1

#print(balanced_accuracy_score(labels, preds))
print(balanced_accuracy_score(labels1, preds1))
print(balanced_accuracy_score(y_val, val_preds))

#print(confusion_matrix(labels, preds))
print(confusion_matrix(labels1, preds1))
print(confusion_matrix(y_val, val_preds))

0.6386020651310564
0.7499922902214611
[[707 552]
 [358 901]]
[[2341 1851]
 [ 246 3962]]


MODEL SAVING

In [159]:
import pickle as pk

filepath="../models/xgbmodel"
pk.dump(model, open(filepath, 'wb'))

FEATURE IMPROTANCE PLOT

In [ ]:
from matplotlib import pyplot as plt

# plot feature importance
xgb.plot_importance(model)
plt.gcf().set_size_inches(20, 20)

plt.show()